# 3. [Trivy](https://github.com/knqyf263/trivy) 編: curl で Docker イメージを脆弱性スキャン

Trivy で脆弱性スキャンしてみましょう！

## 3.1. ソフトウェアとそのアーキテクチャについて

### Trivy

[CIで使えるコンテナの脆弱性スキャナ - Qiita](https://qiita.com/knqyf263/items/dc179f9223fc31b5a51c)  
Clair と違いデータベースもローカルにもち、CI での実行を想定した CLI ツール。  
AlpineLinux については Clair よりずっと正確な検査ができるため  
今回は比較対象として Trivy も試してみます。

### Trivy の REST API 化

Trivy は CLI ツールであるため通常は CI から利用するのですが、ECS でホストするために [Web サーバ](https://github.com/pottava/trivy-restapi)でラップしています。これにより、CI で都度脆弱性データベースの作成から始まってしまうことも回避できます。

## 3.2. 一般 Docker イメージのスキャン

Clair では脆弱性なしと結果がでた `envoyproxy/envoy-alpine:v1.10.0` を Trivy で検査してみましょう。

In [ ]:
source ~/config/.env

curl -s -X GET -H 'Content-Type:application/json' \
  "http://${API_HOST}:9000/api/v1/images/envoyproxy%2Fenvoy-alpine%3Av1.10.0/vulnerabilities" \
  | jq .

成功時応答例）
```text
{
  "Count": 1,
  "Vulnerabilities": [
    {
      "Description": "ChaCha20-Poly1305 is an AEAD cipher, ..."
      "FixedVersion": "1.1.1b-r1",
      "InstalledVersion": "1.1.1a-r1",
      "PkgName": "openssl",
      "References": [
        "https://git.openssl.org/gitweb/?p=openssl.git;a=commitdiff;",
        ..
      ],
      "Severity": "MEDIUM",
      "Title": "openssl: ChaCha20-Poly1305 with long nonces",
      "VulnerabilityID": "CVE-2019-1543"
    }
  ]
}
```

Trivy では一件脆弱性が指摘されましたね。Alpine Linux に対しては評判通り強そうです。

同じ Docker 公式のイメージであっても、古いバージョンには脆弱性が残っている例もみてみましょう。

In [ ]:
curl -s -X GET -H 'Content-Type:application/json' \
  "http://${API_HOST}:9000/api/v1/images/python%3A3.7.3-alpine3.9/vulnerabilities" \
  | jq .

In [ ]:
curl -s -X GET -H 'Content-Type:application/json' \
  "http://${API_HOST}:9000/api/v1/images/python%3A3.4.10-alpine3.9/vulnerabilities" \
  | jq .

## 3.3. CI におけるリリース判定

CI の中で、脆弱な Docker イメージはリリースを停止するのが一般的な流れかと思います。  
Clair + Klar であっても Trivy であっても、**exit code** や **応答値** をみて  
処理が停止するように書くことができます。

Clair なら

In [ ]:
docker run --rm \
      -e CLAIR_ADDR="http://${API_HOST}:6060" \
      supinf/klar:2.4 envoyproxy/envoy:v1.10.0 \
      | jq ".Vulnerabilities | length"

Trivy なら

In [ ]:
curl -s -X GET -H 'Content-Type:application/json' \
  "http://${API_HOST}:9000/api/v1/images/python%3A3.4.10-alpine3.9/vulnerabilities?skip-update=yes" \
  | jq -r ".Count"

といった結果を返すことでエラー判定をすることができるかと思います。  
エラー内容をローカルで再現させるためにも、Docker になっていると便利ですね！

[次へ: 04-teardown-resources](04-teardown-resources.ipynb)